In [1]:
# Weights and Biases related imports
import wandb
from wandb.keras import WandbMetricsLogger

In [2]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_cifar10_batch(file_path):
    with open(file_path, 'rb') as file:
        batch = pickle.load(file, encoding='bytes')
    return batch

def load_cifar10_data(folder_path):
    train_data = []
    train_labels = []

    for i in range(1, 6):
        batch_file = f"{folder_path}/data_batch_{i}"
        batch = load_cifar10_batch(batch_file)
        train_data.append(batch[b'data'])
        train_labels.extend(batch[b'labels'])

    test_batch_file = f"{folder_path}/test_batch"
    test_batch = load_cifar10_batch(test_batch_file)
    test_data = test_batch[b'data']
    test_labels = test_batch[b'labels']

    train_data = np.vstack(train_data)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels

def preprocess_data(train_data, train_labels, test_data, test_labels):
    train_data = train_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    test_data = test_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

    train_labels_onehot = to_categorical(train_labels)
    test_labels_onehot = to_categorical(test_labels)

    return train_data, train_labels_onehot, test_data, test_labels_onehot

cifar10_folder = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(cifar10_folder)

x_train, y_train, x_test, y_test = preprocess_data(
    train_data, train_labels, test_data, test_labels
)

print("Train Data Shape:", x_train.shape)
print("Train Labels Shape:", y_train.shape)
print("Test Data Shape:", x_test.shape)
print("Test Labels Shape:", y_test.shape)

Train Data Shape: (50000, 32, 32, 3)
Train Labels Shape: (50000, 10)
Test Data Shape: (10000, 32, 32, 3)
Test Labels Shape: (10000, 10)


In [3]:
# os.environ['WANDB_NOTEBOOK_NAME'] = 'RUN_1'
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lihem (takim). Use `wandb login --relogin` to force relogin


True

In [4]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'learning_rate': {
        'values': np.logspace(-6, -3, num=6).tolist()
        },
    'batch_size': {
          'values': [64, 128]
        },
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'earlystopping_patience': {
        'value': 10},
    'epochs': {
        'value': 100}
    })

In [5]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'batch_size': {'values': [64, 128]},
                'earlystopping_patience': {'value': 10},
                'epochs': {'value': 100},
                'learning_rate': {'values': [1e-06,
                                             3.981071705534969e-06,
                                             1.584893192461114e-05,
                                             6.309573444801929e-05,
                                             0.00025118864315095795,
                                             0.001]}}}


In [6]:
sweep_id = wandb.sweep(sweep_config, project="CIFAR-10_Classification")

Create sweep with ID: k7rp7vr5
Sweep URL: https://wandb.ai/takim/CIFAR-10_Classification/sweeps/k7rp7vr5


In [7]:
import tensorflow as tf

# Define the VGG16 architecture
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(units=4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(units=4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

In [8]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def train(config = None):
    with wandb.init(config=config):

        config = wandb.config

        tf.keras.backend.clear_session()
        model = create_model()
        model.compile(
            optimizer = Adam(learning_rate=config["learning_rate"]),
            loss = "categorical_crossentropy",
            metrics = ["accuracy", tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top@3_accuracy')]
        )

        early_stopping = EarlyStopping(monitor='val_loss',
                                    patience=config["earlystopping_patience"],
                                    restore_best_weights=True)

        model.fit(x_train, y_train,
                                    epochs=config["epochs"],
                                    batch_size=config["batch_size"],
                                    validation_split=0.1,
                                    callbacks=[
                                        WandbMetricsLogger(log_freq='epoch'),
                                        early_stopping
                                    ], verbose=1
                                    )

In [9]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: pk5no84j with config:
wandb: 	batch_size: 64
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	learning_rate: 1e-06
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 26s 30ms/step - loss: 2.2636 - accuracy: 0.1402 - top@3_accuracy: 0.3847 - val_loss: 2.1295 - val_accuracy: 0.2348 - val_top@3_accuracy: 0.5240
Epoch 2/100
704/704 [==============================] - 20s 29ms/step - loss: 2.0515 - accuracy: 0.2246 - top@3_accuracy: 0.5450 - val_loss: 1.9447 - val_accuracy: 0.2742 - val_top@3_accuracy: 0.6078
Epoch 3/100
704/704 [==============================] - 19s 27ms/step - loss: 1.9305 - accuracy: 0.2727 - top@3_accuracy: 0.6130 - val_loss: 1.8352 - val_accuracy: 0.3126 - val_top@3_accuracy: 0.6676
Epoch 4/100
704/704 [==============================] - 19s 27ms/step - loss: 1.8338 - accuracy: 0.3108 - top@3_accuracy: 0.6564 - val_loss: 1.7473 - val_accuracy: 0.3462 - val_top@3_accuracy: 0.6946
Epoch 5/100
704/704 [==============================] - 20s 28ms/step - loss: 1.7619 - accuracy: 0.3371 - top@3_accuracy: 0.6854 - val_loss: 1.7085 - val_accuracy: 0.3664 - val_top@3_accuracy: 0.7104
Epoch

epoch/accuracy,▁▃▃▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▆▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
epoch/top@3_accuracy,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████
epoch/val_accuracy,▁▂▃▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
epoch/val_loss,█▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁
epoch/val_top@3_accuracy,▁▄▅▅▆▆▇▇▇▇▇▇▇▇▇▇█▇███▇██████████████████
epoch/accuracy,0.72769
epoch/epoch,78
epoch/learning_rate,0.0


wandb: Agent Starting Run: 35zippcz with config:
wandb: 	batch_size: 64
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	learning_rate: 3.981071705534969e-06
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 20s 28ms/step - loss: 2.0615 - accuracy: 0.2051 - top@3_accuracy: 0.5136 - val_loss: 1.7396 - val_accuracy: 0.3362 - val_top@3_accuracy: 0.7000
Epoch 2/100
704/704 [==============================] - 19s 27ms/step - loss: 1.7009 - accuracy: 0.3570 - top@3_accuracy: 0.7102 - val_loss: 1.5859 - val_accuracy: 0.4022 - val_top@3_accuracy: 0.7632
Epoch 3/100
704/704 [==============================] - 19s 27ms/step - loss: 1.5445 - accuracy: 0.4253 - top@3_accuracy: 0.7794 - val_loss: 1.4356 - val_accuracy: 0.4684 - val_top@3_accuracy: 0.8180
Epoch 4/100
704/704 [==============================] - 19s 27ms/step - loss: 1.4472 - accuracy: 0.4639 - top@3_accuracy: 0.8063 - val_loss: 1.3539 - val_accuracy: 0.5020 - val_top@3_accuracy: 0.8328
Epoch 5/100
704/704 [==============================] - 19s 27ms/step - loss: 1.3739 - accuracy: 0.4943 - top@3_accuracy: 0.8245 - val_loss: 1.2875 - val_accuracy: 0.5280 - val_top@3_accuracy: 0.8454
Epoch

...

epoch/accuracy,▁▂▄▅▆▆▇▇▇█████████
epoch/epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁
epoch/top@3_accuracy,▁▄▆▇▇▇████████████
epoch/val_accuracy,▁▃▆▇▇▇████████████
epoch/val_loss,█▆▃▂▁▁▁▁▁▁▁▂▂▂▃▃▃▃
epoch/val_top@3_accuracy,▁▅▇▇██████████████
epoch/accuracy,0.97262
epoch/epoch,17
epoch/learning_rate,0.00025


wandb: Agent Starting Run: jx4evlsg with config:
wandb: 	batch_size: 128
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	learning_rate: 0.001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
352/352 [==============================] - 16s 44ms/step - loss: 2.3682 - accuracy: 0.0997 - top@3_accuracy: 0.2972 - val_loss: 2.3028 - val_accuracy: 0.1024 - val_top@3_accuracy: 0.2960
Epoch 2/100
352/352 [==============================] - 15s 42ms/step - loss: 2.3141 - accuracy: 0.1003 - top@3_accuracy: 0.2985 - val_loss: 2.3033 - val_accuracy: 0.0958 - val_top@3_accuracy: 0.2884
Epoch 3/100
352/352 [==============================] - 15s 43ms/step - loss: 2.3029 - accuracy: 0.0982 - top@3_accuracy: 0.2979 - val_loss: 2.3028 - val_accuracy: 0.0976 - val_top@3_accuracy: 0.2990
Epoch 4/100
352/352 [==============================] - 15s 43ms/step - loss: 2.3028 - accuracy: 0.1005 - top@3_accuracy: 0.2986 - val_loss: 2.3029 - val_accuracy: 0.0950 - val_top@3_accuracy: 0.2884
Epoch 5/100
352/352 [==============================] - 15s 43ms/step - loss: 2.3028 - accuracy: 0.0978 - top@3_accuracy: 0.2956 - val_loss: 2.3027 - val_accuracy: 0.0958 - val_top@3_accuracy: 0.2952
Epoch

epoch/accuracy,▆▇▃█▂▅▅▅▆▅▆▅▃▇▅▃▁█
epoch/epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/top@3_accuracy,▆▇▆▇▄▂▇▃▅▅▅▄██▄▄▁█
epoch/val_accuracy,█▂▃▁▂▂█▃▁▁▂▃▁▁▂▃▂▃
epoch/val_loss,▃█▃▄▁▃▄▁▃▄▂▃▃▄▃▃▃▃
epoch/val_top@3_accuracy,▆▁█▁▆▆▆█▁▂▁▃▁▁▃▄▁▄
epoch/accuracy,0.1006
epoch/epoch,17
epoch/learning_rate,0.001


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
